In [114]:
import vincent 
import pandas as pd
import random
import json
import numpy as np
vincent.core.initialize_notebook()
# run `git clone git@github.com:wrobstory/vincent_map_data.git` in this directory

In [115]:
with open('vincent_map_data/us_counties.topo.json', 'r') as f:
    get_id = json.load(f)

In [116]:
new_geoms = []
for geom in get_id['objects']['us_counties.geo']['geometries']:
    geom['properties']['FIPS'] = int(geom['properties']['FIPS'])
    new_geoms.append(geom)

get_id['objects']['us_counties.geo']['geometries'] = new_geoms
with open('us_counties.topo.json', 'w') as f:
    json.dump(get_id, f)

In [117]:
geometries = get_id['objects']['us_counties.geo']['geometries']
county_codes = [x['properties']['FIPS'] for x in geometries]
county_df = pd.DataFrame({'FIPS': county_codes}, dtype=str)
county_df = county_df.astype(int)

In [120]:
geo_data = [{'name': 'counties',
             'url': 'us_counties.topo.json',
             'feature': 'us_counties.geo'},
            {'name': 'states',
             'url': 'vincent_map_data/us_states.topo.json',
             'feature': 'us_states.geo'}
           ]

In [121]:
li = []
# see email
with open('NRDC_county_totals.csv', 'r') as csv:
    previous_value = ''
    for line in csv.readlines():
        split = line.split(',')
        if split[0] == '':
            split[0] = previous_value
        else:
            previous_value = split[0]
        numzeros = 3 - len(split[1])
        l = ['0'] * numzeros
        s = split[0] + "".join(l) + split[1]
        li.append([s, float(split[2].replace('\n', ''))])        

In [122]:
df = pd.DataFrame(li)

In [123]:
df.columns = ['FIPS', 'TOV']

In [125]:
df['LOG_TOV'] = np.log(df['TOV'])

In [126]:
geometries = get_id['objects']['us_counties.geo']['geometries']
county_codes = [x['properties']['FIPS'] for x in geometries]
county_df = pd.DataFrame({'FIPS': county_codes}, dtype=str)
county_df = county_df.astype(str)
merged = pd.merge(df, county_df, on="FIPS", how="outer")

In [128]:
geo_data = [{'name': 'counties',
             'url': 'us_counties.topo.json',
             'feature': 'us_counties.geo'},
            {'name': 'states',
             'url': 'vincent_map_data/us_states.topo.json',
             'feature': 'us_states.geo'}
           ]

vis = vincent.Map(geo_data=geo_data, data=merged, scale=1000, projection='albersUsa',
                data_bind='TOV', data_key='FIPS',
                map_key={'counties': 'properties.FIPS'})


In [130]:
vis.rebind(column='TOV', brew='Blues')
del vis.marks[1].properties.update
vis.marks[1].properties.enter.stroke.value = '#c4c4c4'
vis.marks[0].properties.enter.stroke.value = '#c4c4c4'
vis.marks[0].properties.enter.stroke_width = vincent.ValueRef(value=1)
vis.marks[0].properties.enter.stroke_opacity = vincent.ValueRef(value=.2)

In [131]:
vis.display()